# Task 4: Portfolio Optimization
# 
In this notebook, we use the forecasted return for TSLA (from our best model) and historical returns for BND and SPY to construct an optimal portfolio using Modern Portfolio Theory (MPT). We will:
 - Calculate expected returns and the covariance matrix.
 - Generate the Efficient Frontier.
 - Identify the Maximum Sharpe Ratio and Minimum Volatility portfolios.
 - Visualize the frontier and recommend an optimal allocation.
 
# 
#  1. Setup and Imports


In [ ]:
# Task 4: Portfolio Optimization using Forecasted and Historical Returns

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pypfopt import EfficientFrontier, risk_models, expected_returns, plotting


## 2. Load and Prepare Data
# 
 We load the processed price data for TSLA, BND, and SPY. If you have run the previous notebooks, these DataFrames should be available. Otherwise, load them from CSV or your data loader.


In [ ]:

# Error handling for missing data
try:
    tsla_processed = pd.read_csv('../data/TSLA_processed.csv', index_col=0, parse_dates=True)
except FileNotFoundError:
    print("TSLA data file not found. Please run the data loader notebook first.")
# Load processed price data for TSLA, BND, SPY

tsla_processed = pd.read_csv('../data/TSLA_processed.csv', index_col=0, parse_dates=True)
bnd_processed = pd.read_csv('../data/BND_processed.csv', index_col=0, parse_dates=True)
spy_processed = pd.read_csv('../data/SPY_processed.csv', index_col=0, parse_dates=True)

# For demonstration, let's assume you have these DataFrames:
# tsla_processed['Close'], bnd_processed['Close'], spy_processed['Close']

assets = ['TSLA', 'BND', 'SPY']
prices = pd.DataFrame({
    'TSLA': tsla_processed['Close'],
    'BND': bnd_processed['Close'],
    'SPY': spy_processed['Close']
})

# Calculate daily returns
returns = prices.pct_change().dropna()


 ## 3. Set Expected Returns
# 
- For TSLA, we use the forecasted annualized return from our best model's 12-month forecast.
- For BND and SPY, we use historical annualized mean returns.


In [ ]:

# Use forecasted annualized return for TSLA (from your best model's 12-month forecast)
tsla_current = tsla_processed['Close'].iloc[-1]
tsla_forecast_end = future_forecast.iloc[-1]
tsla_expected_annual_return = (tsla_forecast_end / tsla_current) ** (252/len(future_forecast)) - 1

# Use historical annualized mean returns for BND and SPY
bnd_expected_annual_return = returns['BND'].mean() * 252
spy_expected_annual_return = returns['SPY'].mean() * 252

mu = pd.Series({
    'TSLA': tsla_expected_annual_return,
    'BND': bnd_expected_annual_return,
    'SPY': spy_expected_annual_return
})

print("Expected annual returns:")
display(mu)


## 4. Covariance Matrix of Returns


In [ ]:
S = risk_models.sample_cov(returns)
print("Covariance matrix of returns:")
display(S)

## 5. Efficient Frontier Optimization
# 
We use the PyPortfolioOpt library to:
- Find the portfolio with the maximum Sharpe ratio (risk-adjusted return).
- Find the portfolio with the minimum volatility.
- Display the optimal weights and performance metrics.


In [ ]:
# Efficient Frontier: Max Sharpe Ratio Portfolio
ef = EfficientFrontier(mu, S)
ef.add_constraint(lambda w: w <= 0.6)  # Max 60% per asset
ef.add_constraint(lambda w: w >= 0)    # No shorting
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print("Optimal Weights (Max Sharpe):", cleaned_weights)
ret, vol, sharpe = ef.portfolio_performance(verbose=True)

# Minimum Volatility Portfolio
ef_minvol = EfficientFrontier(mu, S)
minvol_weights = ef_minvol.min_volatility()
minvol_cleaned = ef_minvol.clean_weights()
print("Optimal Weights (Min Volatility):", minvol_cleaned)
ret_minvol, vol_minvol, sharpe_minvol = ef_minvol.portfolio_performance(verbose=True)


## 6. Visualize the Efficient Frontier


In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
ax.set_title("Efficient Frontier: TSLA, BND, SPY")
plt.show()


## 7. Compare Portfolio Weights


In [ ]:
weights_df = pd.DataFrame({
    'Max Sharpe': pd.Series(cleaned_weights),
    'Min Volatility': pd.Series(minvol_cleaned)
})
print("Portfolio Weights Comparison:")
display(weights_df)

## 8. Recommendation and Summary
# 
Based on the analysis, we recommend the Max Sharpe portfolio for its superior risk-adjusted return. Below are the recommended weights and expected performance metrics.


In [ ]:
print("\nRecommended Portfolio (Max Sharpe):")
for asset, weight in cleaned_weights.items():
    print(f"{asset}: {weight:.2%}")

print(f"\nExpected annual return: {ret:.2%}")
print(f"Annual volatility: {vol:.2%}")
print(f"Sharpe Ratio: {sharpe:.2f}")


# Save weights for use in backtesting
weights_df.to_csv('../results/forecasting/optimal_portfolio_weights.csv')



### **Conclusion**
# 
We have constructed an optimal portfolio using forecasted and historical returns, visualized the efficient frontier, and recommended an allocation that balances risk and return. In the next task, we will backtest this strategy to evaluate its real-world performance.